# Домашнее задание 2. Классификация изображений.

## Вступление

В этом задании потребуется обучить классификатор изображений. Будем работать с датасетом, название которого раскрывать не будем. Можете посмотреть самостоятельно на картинки, которые в есть датасете. В нём 200 классов и около 5 тысяч картинок на каждый класс. Классы пронумерованы, как нетрудно догадаться, от 0 до 199. Скачать датасет можно вот [тут](https://yadi.sk/d/BNR41Vu3y0c7qA).

Структура датасета простая -- есть директории train/ и val/, в которых лежат обучающие и валидационные данные. В train/ и val/ лежат директориии, соответствующие классам изображений, в которых лежат, собственно, сами изображения.
 
__Задание__. Необходимо выполнить два задания

1) Добейтесь accuracy **на валидации не менее 0.44**. В этом задании **запрещено** пользоваться предобученными моделями и ресайзом картинок. 5 баллов

2) Добейтесь accuracy **на валидации не менее 0.84**. В этом задании делать ресайз и использовать претрейн можно. 5 баллов

Напишите краткий отчёт о проделанных экспериментах. Что сработало и что не сработало? Почему вы решили, сделать так, а не иначе? Обязательно указывайте ссылки на чужой код, если вы его используете. Обязательно ссылайтесь на статьи / блогпосты / вопросы на stackoverflow / видосы от ютуберов-машинлернеров / курсы / подсказки от Дяди Васи и прочие дополнительные материалы, если вы их используете. 

Ваш код обязательно должен проходить все `assert`'ы ниже.

__Использовать внешние данные для обучения строго запрещено в обоих заданиях. Также запрещено обучаться на валидационной выборке__.


__Критерии оценки__: Оценка вычисляется по простой формуле: `min(10, 10 * Ваша accuracy / 0.44)` для первого задания и `min(10, 10 * (Ваша accuracy - 0.5) / 0.34)` для второго. Оценка округляется до десятых по арифметическим правилам.


__Советы и указания__:
 - Наверняка вам потребуется много гуглить о классификации и о том, как заставить её работать. Это нормально, все гуглят. Но не забывайте, что нужно быть готовым за скатанный код отвечать :)
 - Используйте аугментации. Для этого пользуйтесь модулем `torchvision.transforms` или библиотекой [albumentations](https://github.com/albumentations-team/albumentations)
 - Можно обучать с нуля или файнтюнить (в зависимости от задания) модели из `torchvision`.
 - Рекомендуем написать вам сначала класс-датасет (или воспользоваться классом `ImageFolder`), который возвращает картинки и соответствующие им классы, а затем функции для трейна по шаблонам ниже. Однако делать это мы не заставляем. Если вам так неудобно, то можете писать код в удобном стиле. Однако учтите, что чрезмерное изменение нижеперечисленных шаблонов увеличит количество вопросов к вашему коду и повысит вероятность вызова на защиту :)
 - Валидируйте. Трекайте ошибки как можно раньше, чтобы не тратить время впустую.
 - Чтобы быстро отладить код, пробуйте обучаться на маленькой части датасета (скажем, 5-10 картинок просто чтобы убедиться что код запускается). Когда вы поняли, что смогли всё отдебажить, переходите обучению по всему датасету
 - На каждый запуск делайте ровно одно изменение в модели/аугментации/оптимайзере, чтобы понять, что и как влияет на результат.
 - Фиксируйте random seed.
 - Начинайте с простых моделей и постепенно переходите к сложным. Обучение лёгких моделей экономит много времени.
 - Ставьте расписание на learning rate. Уменьшайте его, когда лосс на валидации перестаёт убывать.
 - Советуем использовать GPU. Если у вас его нет, используйте google colab. Если вам неудобно его использовать на постоянной основе, напишите и отладьте весь код локально на CPU, а затем запустите уже написанный ноутбук в колабе. Авторское решение задания достигает требуемой точности в колабе за 15 минут обучения.
 
Good luck & have fun! :)

In [ ]:
import os
import abc
import time
import typing
import random
import warnings

import PIL
import tqdm
import torch
import wandb
import numpy
import pandas
import torchscan
import torchvision
import sklearn.metrics
import matplotlib.pyplot as plt
import torch.utils.data as torchdata
from torchvision.transforms import v2 as transforms

device = torch.device(
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(device)

RANDOM_STATE = 42
def set_random_seed(seed):
    random.seed(seed)
    numpy.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
def fix_random():
    return set_random_seed(RANDOM_STATE)
fix_random()

In [ ]:
wandb.login(anonymous = "allow")

**✨ Внимание ✨**

В этом домашнем задании предлагается использовать библиотеку `pytorch_lightning`. Доступ к ее [документации](https://lightning.ai/docs/pytorch/stable/) заблокирован с территории РФ. Вы можете:

1. Получить к ней доступ с помощью VPN.

2. Собрать документацию самостоятельно. Для этого склонируйте [github-репозиторий](https://github.com/Lightning-AI/lightning/tree/master), запустите в нем терминал (на windows – git bash) и выполните команды:

```shell
git submodule update --init --recursive
make docs
```
После этого откройте появившийся файл `docs/build/html/index.html`. Для работы команд в вашем окружении должен быть `pip`. Полная инструкция [по ссылке](https://github.com/Lightning-AI/lightning/tree/master/docs).

3. Гуглить `<error message> pytorch lightning` или `<how to do this> pytorch lightning`. Stack overflow на территории РФ все еще доступен 😉

4. Не пользоваться `pytorch_lightning` и написать цикл обучения модели самостоятельно. Например, по аналогии с функцией `fit` из [семинара 4](https://github.com/hse-ds/iad-deep-learning/blob/master/2023/seminars/04.%20Optim%20%26%20Lightning/04_Optim%26Lightning_solution.ipynb).

## Задание 0

### Что поможет сделать на 10 из 10 (одно задание - 5 баллов)

1. Использовать все возможные методы оптимизации и эксперемнтировать с ними.
2. Подбор learning rate. Пример из прошлого семинара как это делать: [Как найти lr](https://pytorch-lightning.readthedocs.io/en/1.4.5/advanced/lr_finder.html)

```
  trainer = pl.Trainer(accelerator="gpu", max_epochs=2, auto_lr_find=True) 

  trainer.tune(module, train_dataloader, eval_dataloader)

  trainer.fit(module, train_dataloader, eval_dataloader))
```



3. Аугментация данных. [Документация (полезная)](https://pytorch.org/vision/main/transforms.html), а также [библиотека albumentation](https://towardsdatascience.com/getting-started-with-albumentation-winning-deep-learning-image-augmentation-technique-in-pytorch-47aaba0ee3f8)
4. Подбор архитектуры модели. 
5. Можно написать модель руками свою в YourNet, а можно импортировать не предобученную сетку известной архитектуры из модуля torchvision.models. Один из способов как можно сделать: 

  * `torchvision.models.resnet18(pretrained=False, num_classes=200).to(device)`
  * Документация по возможным моделям и как их можно брать: [Документация (полезная)](https://pytorch.org/vision/stable/models.html)
6. Правильно нормализовывать данные при создании, пример [тык, но тут и в целом гайд от и до](https://www.pluralsight.com/guides/image-classification-with-pytorch)
7. Model Checkpointing. Сохраняйте свой прогресс (модели), чтобы когда что-то пойдет не так вы сможете начать с этого места или просто воспроизвести свои результаты модели, которые обучали. 
 * Пример как можно с wandb тут: [Сохраняем лучшие модели в wandb](https://docs.wandb.ai/guides/integrations/lightning)
 * По простому можно так: [Сохраняем модели в pytorch дока](https://pytorch.org/tutorials/beginner/saving_loading_models.html)

### Подготовка данных

In [ ]:
class MysteriousDataset(torchdata.Dataset):
    def __init__(
            self,
            train: bool,
            preload: bool = True,
            precalculate_transform: bool = True,
            transform: typing.Optional[transforms.Compose] = None
        ):
        # Create a way to easily read from disk
        self.name = "train" if train else "val"
        self.dataset_src = "./dataset/{}".format(self.name)
        self.dataset = torchvision.datasets.ImageFolder(self.dataset_src)

        self.classes = self.dataset.classes
        self.precalculated_transform = None
        self.transform = None

        # The dataset is not that big, so we might want to load it into RAM beforehand
        if preload or precalculate_transform:
            if precalculate_transform:
                # We might also want to precalculate the transform
                self.precalculated_transform = transform
                self.transform = transform
                transform = None
            # If precalculate_transform is True, self.transform is set, and __getitem__ will apply the transforms
            self.images, self.targets = self.load_all("Preload {}".format(self.name))
        # If precalculate_transform is True, this will be None
        self.transform = transform

    def load_all(self, progress_bar: bool = False):
        images = [ ]
        targets = [ ]
        for record in (tqdm.tqdm(self, desc = progress_bar) if progress_bar else self):
            images.append(record[0])
            targets.append(record[1])
        # If no transforms are applied, torch.stack(images) will fail
        try: return torch.stack(images), targets
        except: return images, targets

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        if hasattr(self, 'images') and hasattr(self, 'targets'):
            # If the data has been preloaded, use it
            image, target = self.images[idx], self.targets[idx]
        else:
            # Access the source
            image, target = self.dataset[idx]
        # Apply the transform if needed
        if self.transform is not None:
            image = self.transform(image)
        return image, target
    
    def channel_stats(self):
        images, _ = self.load_all() # Get the dataset as two tensors
        # Calculate the metrics
        mean = torch.mean(images, dim = [0, 2, 3])
        std = torch.std(images, dim = [0, 2, 3])
        return mean, std

In [ ]:
transform = torchvision.transforms.v2.Compose([
    torchvision.transforms.v2.ToImage(),
    torchvision.transforms.v2.ToDtype(torch.float32, scale = True)
])

no_preload = MysteriousDataset(True, transform = transform, preload = False, precalculate_transform = False)
no_precalc = MysteriousDataset(True, transform = transform, preload = True, precalculate_transform = False)
preload = MysteriousDataset(True, transform = transform, preload = True, precalculate_transform = True)
torch_dataset = torchvision.datasets.ImageFolder("./dataset/train", transform = transform)

In [ ]:
# Check that it is actually faster to loop through the dataset when it is preloaded into RAM
def test_loading(*args):
    for dataset, desc in args:
        loader = torchdata.DataLoader(dataset, batch_size = 256)
        for item in tqdm.tqdm(loader, desc = desc):
            pass

test_loading(
    (torch_dataset, 'ImageFolder'),
    (no_preload, 'no_preload'),
    (no_precalc, 'no_precalc'),
    (preload, 'preload')
)

#### Посчитаем поканальные средние и стандартные отклонения для нормализации данных

In [ ]:
mean, std = preload.channel_stats()
print(mean, std)

In [ ]:
del torch_dataset
del no_preload
del no_precalc
del preload

#### Создадим датасеты

In [ ]:
fix_random()

# Transforms
transform = torchvision.transforms.v2.Compose([
    torchvision.transforms.v2.ToImage(),
    torchvision.transforms.v2.ToDtype(torch.float32, scale = True),
    torchvision.transforms.v2.Normalize(mean, std)
])

# Load datasets
train_set = MysteriousDataset(train = True, preload = True, precalculate_transform = True, transform = transform)
test_set = MysteriousDataset(train = False, preload = True, precalculate_transform = True, transform = transform)

# Check
print(len(train_set), len(test_set))

# Just very simple sanity checks
assert isinstance(train_set[0], tuple)
assert len(train_set[0]) == 2
assert isinstance(train_set[1][1], int)
assert isinstance(train_set[1][0], torch.Tensor)
assert train_set[1][0].shape == torch.Size([ 3, 64, 64 ])
print("Dataset tests passed")

for images, targets in torchdata.DataLoader(train_set, batch_size = 256):
    assert isinstance(images, torch.Tensor)
    assert isinstance(targets, torch.Tensor)
    assert images.shape == torch.Size([ 256, 3, 64, 64 ])
    assert targets.shape == torch.Size([ 256 ])
    print("DataLoader tests passed")
    break

### Посмотрим на картиночки

In [ ]:
fix_random()

# Denormalization
denormalize = torchvision.transforms.v2.Compose([
    torchvision.transforms.v2.Normalize(mean = [ 0., 0., 0. ], std = 1 / std),
    torchvision.transforms.v2.Normalize(mean = -mean, std = [ 1., 1., 1. ])
])

# Display some samples from each dataset
def display_examples(dataset: MysteriousDataset, row: int):
    train_loader = torchdata.DataLoader(dataset, batch_size = 10, shuffle = True)
    for i, (image, label) in enumerate(zip(*next(iter(train_loader)))):
        plt.subplot(3, 10, i + 10 * (row - 1) + 1)
        plt.axis('off')
        plt.title('{}'.format(label))
        plt.imshow((denormalize(image).permute(1, 2, 0).numpy() * 255).astype(numpy.uint8))

plt.rcParams["figure.figsize"] = (15, 5)
display_examples(train_set, 1)
display_examples(test_set, 2)

## Задание 1. 

5 баллов
Добейтесь accuracy на валидации не менее 0.44. В этом задании запрещено пользоваться предобученными моделями и ресайзом картинок.


Для того чтобы выбить скор (считается ниже) на 2.5/5 балла (то есть половину за задание) достаточно соблюдать пару простых жизненных правил:
1. Аугментация (без нее сложно очень будет)
2. Оптимайзеры можно (и нужно) использовать друг с другом. Однако когда что-то проверяете, то не меняйте несколько параметров сразу - собьете логику экспериментов
3. Не используйте полносвязные модели или самые первые сверточные, используйте более современные архитектуры (что на лекциях встречались)
4. Посмотреть все ноутбуки прошедших семинаров и слепить из них что-то общее. Семинарских тетрадок хватит сверх

### Классификатор

In [ ]:
class BaseClassifier(abc.ABC):
    @abc.abstractmethod
    def fit(self, train_set: torchdata.Dataset, val_set: torchdata.Dataset):
        raise NotImplementedError

    @abc.abstractmethod
    def predict(self, images: torch.Tensor) -> typing.Tuple[torch.Tensor, torch.Tensor]:
        raise NotImplementedError
    
    def calc_metrics(self, dataset: torchdata.Dataset) -> dict:
        num_classes = self.output_shape[0] if hasattr(self, 'output_shape') else len(dataset.classes)
        classes = torch.arange(num_classes)

        all_labels = torch.tensor([])
        all_predictions = torch.tensor([])
        all_scores = torch.empty((0, num_classes))
        loader = torchdata.DataLoader(dataset, batch_size = 512, shuffle = False)
        for images, labels in loader:
            predictions, scores = self.predict(images)
            all_labels = torch.cat([ all_labels, labels ])
            all_scores = torch.cat([ all_scores, scores.detach().cpu() ])
            all_predictions = torch.cat([ all_predictions, predictions.detach().cpu() ])

        return {
            'Accuracy':       sklearn.metrics.accuracy_score      (all_labels, all_predictions),
            'TOP-2 Accuracy': sklearn.metrics.top_k_accuracy_score(all_labels, all_scores, k = 2, labels = classes),
            'TOP-3 Accuracy': sklearn.metrics.top_k_accuracy_score(all_labels, all_scores, k = 3, labels = classes),
            'TOP-4 Accuracy': sklearn.metrics.top_k_accuracy_score(all_labels, all_scores, k = 4, labels = classes),
            'TOP-5 Accuracy': sklearn.metrics.top_k_accuracy_score(all_labels, all_scores, k = 5, labels = classes),
            'TOP-6 Accuracy': sklearn.metrics.top_k_accuracy_score(all_labels, all_scores, k = 6, labels = classes),
            'TOP-7 Accuracy': sklearn.metrics.top_k_accuracy_score(all_labels, all_scores, k = 7, labels = classes),
            'TOP-8 Accuracy': sklearn.metrics.top_k_accuracy_score(all_labels, all_scores, k = 8, labels = classes),
            'TOP-9 Accuracy': sklearn.metrics.top_k_accuracy_score(all_labels, all_scores, k = 9, labels = classes),
            # 'AUC-ROC':        sklearn.metrics.roc_auc_score       (all_labels, all_scores, multi_class = 'ovo'),
            'Precision':      sklearn.metrics.precision_score     (all_labels, all_predictions, average = 'macro', zero_division = 0),
            'Recall':         sklearn.metrics.recall_score        (all_labels, all_predictions, average = 'macro', zero_division = 0),
            'F1-score':       sklearn.metrics.f1_score            (all_labels, all_predictions, average = 'macro', zero_division = 0)
        }

In [ ]:
class Classifier(BaseClassifier):
    results = [ ]

    def __init__(
            self,
            name: str,
            model: torch.nn.Module,
            batch_size: int = 256,
            learning_rate: int = 1e-3,
            device: torch.device = device,
            optimizer: typing.Optional[torch.optim.Optimizer] = None,
            scheduler: typing.Optional[torch.optim.lr_scheduler.LRScheduler] = None,
        ):
        self.name = name
        self.history = [ ]
        self.device = device
        self.input_shape = None
        self.output_shape = None
        self.scheduler = scheduler
        self.batch_size = batch_size
        self.model = model.to(self.device)
        self.optimizer = optimizer or torch.optim.AdamW(self.model.parameters(), lr = learning_rate)


    def train(self, images: torch.Tensor, labels: torch.Tensor) -> float:
        self.model.train() # Enter train mode
        self.optimizer.zero_grad() # Zero gradients
        output = self.model(images.to(self.device)) # Get predictions
        loss = torch.nn.functional.cross_entropy(output, labels.to(self.device)) # Calculate loss
        loss.backward() # Calculate gradients
        self.optimizer.step() # Update weights
        return loss.item()

    def train_epoch(self, loader: torchdata.DataLoader) -> float:
        sum_loss = 0
        for images, labels in loader:
            sum_loss += self.train(images, labels) # Train one batch
        if self.scheduler is not None:
            self.scheduler.step()
        return sum_loss / len(loader) # Return average loss to avoid random-dependent graph
       
    def fit(self, train_set: torchdata.Dataset, val_set: torchdata.Dataset, n_epochs: int = 25, silent: bool = False):
        if self.input_shape is None or self.output_shape is None:
            self.predict(train_set[0][0].unsqueeze(0)) # Initialize lazy layers and in/out shapes
        loader = torchdata.DataLoader(train_set, batch_size = self.batch_size, shuffle = True)

        wandb_settings = { "silent": True, "disable_git": True } if silent else None
        wandb.init(project = "DL-HW-2", name = self.name, anonymous = "allow", settings = wandb_settings)
        wandb.watch(self.model, log = "all")

        for epoch in tqdm.trange(n_epochs, desc = "Fit {}".format(self.name)):
            # Train
            train_start = time.perf_counter()
            loss = self.train_epoch(loader)
            train_time = time.perf_counter() - train_start

            # Validate
            val_start = time.perf_counter()
            metrics = self.calc_metrics(val_set)
            val_time = time.perf_counter() - val_start
            
            # Upload metrics
            metrics['Validation time'] = val_time
            metrics['Train time'] = train_time
            metrics['Loss'] = loss
            wandb.log(metrics)
            metrics['Epoch'] = epoch + 1
            self.history.append(metrics)

        # Finish the run
        wandb.finish(quiet = True)

        # Store best metrics
        self.best_metrics = max(self.history, key = lambda item: item['Accuracy'])
        Classifier.results.append({ 'Name': self.name, **self.best_metrics })
        return self
    

    def predict(self, images: torch.Tensor) -> typing.Tuple[torch.Tensor, torch.Tensor]:
        if self.input_shape is None:
            self.input_shape = images[0].shape # Lazily initialize input shape

        self.model.eval() # Enter evaluation mode
        with torch.no_grad():
            outputs = self.model(images.to(self.device)) # Get outputs
            scores = torch.softmax(outputs, dim = 1) # Make probabilities
            predictions = torch.argmax(scores, dim = 1) # Calculate predictions

        if self.output_shape is None:
            self.output_shape = scores[0].shape # Lazily initialize output shape
        return predictions, scores
    

    def summary(self):
        display(pandas.DataFrame(Classifier.results)) # Print run history
        warnings.filterwarnings("ignore") # Ignore warnings that might be printed by torchscan.summary
        torchscan.summary(self.model.eval(), self.input_shape, receptive_field = True) # Print summary about this model

### Эксперименты

Будем использовать свертки размера 3x3 с отступом 1. Количество фильтров будем увеличивать в два раза с каждой сверткой, начиная с 16. После каждого слоя будем применять нормализацию и макс-пулинг размера 2x2. \
Для собственно классификации создадим два полносвязных слоя с 1024 и 200 нейронами соответственно (выходы второго - ответы модели для каждого класса). Между слоями добавим нормализацию, а во избежание переобучения перед обоими слоями будем использовать Dropout с вероятностью 0.5. \
В качестве нелинейности выберем GELU. В прошлом ДЗ эта функция показала себя немного лучше других, но сильного влияния на качество это не должно оказывать.

Попробуем модели с различным количеством сверточных слоев.

In [ ]:
fix_random()
model = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(16), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(32), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(64), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),

    torch.nn.Flatten(), torch.nn.Dropout(0.5), torch.nn.LazyLinear(1024), torch.nn.BatchNorm1d(1024), torch.nn.GELU(),
    torch.nn.Dropout(0.5), torch.nn.Linear(1024, 200)
)
Classifier('3 convolutions', model).fit(train_set, test_set).summary()

In [ ]:
fix_random()
model = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(16), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(32), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(64), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(128), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),

    torch.nn.Flatten(), torch.nn.Dropout(0.5), torch.nn.LazyLinear(1024), torch.nn.BatchNorm1d(1024), torch.nn.GELU(),
    torch.nn.Dropout(0.5), torch.nn.Linear(1024, 200)
)
Classifier('4 convolutions', model).fit(train_set, test_set).summary()

In [ ]:
fix_random()
model = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(16), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(32), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(64), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(128), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(256), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),

    torch.nn.Flatten(), torch.nn.Dropout(0.5), torch.nn.LazyLinear(1024), torch.nn.BatchNorm1d(1024), torch.nn.GELU(),
    torch.nn.Dropout(0.5), torch.nn.Linear(1024, 200)
)
Classifier('5 convolutions', model).fit(train_set, test_set).summary()

Добавление четвертого слоя заметно улучшило качество модели, но пятый слой лишь немного его понизил. \
Заметим, что поле восприятия пятого слоя - 94 - явно больше размера обрабатываемых картинок (64x64), а третьего слоя - 22 - заметно меньше. Четвёртый слой же "видит" 46 пикселей, что наиболее близко к размеру картинки. Следовательно, можно предоложить, что наилучшего качества достигнет модель с полем восприятия, равным размеру картинки (не больше и не меньше).

Попробуем убрать одно применения пулинга после четвертого слоя. Это понизит поле восприятия до 62.

In [ ]:
fix_random()
model = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(16), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(32), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(64), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(128), torch.nn.GELU(),
    torch.nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(256), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),

    torch.nn.Flatten(), torch.nn.Dropout(0.5), torch.nn.LazyLinear(1024), torch.nn.BatchNorm1d(1024), torch.nn.GELU(),
    torch.nn.Dropout(0.5), torch.nn.Linear(1024, 200)
)
Classifier('Receptive Field 62', model).fit(train_set, test_set).summary()

Действительно, эта модель показывает гораздо более хорошее качество. Попробуем увеличить поле восприятия до 64 добавлением ещё одного свёрточного в начале.

In [ ]:
fix_random()
model = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(16), torch.nn.GELU(),
    torch.nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(32), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(64), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(128), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(256), torch.nn.GELU(),
    torch.nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(512), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),

    torch.nn.Flatten(), torch.nn.Dropout(0.5), torch.nn.LazyLinear(1024), torch.nn.BatchNorm1d(1024), torch.nn.GELU(),
    torch.nn.Dropout(0.5), torch.nn.Linear(1024, 200)
)
Classifier('Receptive Field 64', model).fit(train_set, test_set).summary()

Стало однозначно лучше. Попробуем добавить scheduler, чтобы стабилизировать процесс обучения

In [ ]:
fix_random()
model = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(16), torch.nn.GELU(),
    torch.nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(32), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(64), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(128), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(256), torch.nn.GELU(),
    torch.nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(512), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),

    torch.nn.Flatten(), torch.nn.Dropout(0.5), torch.nn.LazyLinear(1024), torch.nn.BatchNorm1d(1024), torch.nn.GELU(),
    torch.nn.Dropout(0.5), torch.nn.Linear(1024, 200)
).to(device)
optimizer = torch.optim.AdamW(model.parameters())
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.9)
Classifier('Scheduler', model, optimizer = optimizer, scheduler = scheduler).fit(train_set, test_set).summary()

Лучшее качество достигается на 22 эпохе. Возьмём эту модель в качестве итоговой. Обучим заново для валидации результата (к счастью, это занимает не очень много времени)

### Итоговая модель

In [ ]:
fix_random()
model = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(16), torch.nn.GELU(),
    torch.nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(32), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(64), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(128), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(256), torch.nn.GELU(),
    torch.nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(512), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),

    torch.nn.Flatten(), torch.nn.Dropout(0.5), torch.nn.LazyLinear(1024), torch.nn.BatchNorm1d(1024), torch.nn.GELU(),
    torch.nn.Dropout(0.5), torch.nn.Linear(1024, 200)
).to(device)
optimizer = torch.optim.AdamW(model.parameters())
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.9)
final_model = Classifier('Task 1 final', model, optimizer = optimizer, scheduler = scheduler).fit(train_set, test_set, 22)

In [ ]:
accuracy = final_model.calc_metrics(test_set)['Accuracy']
print(f"Accuracy: {accuracy * 100}%")
print(f"Оценка за это задание составит {numpy.clip(10 * accuracy / 0.44, 0, 10):.2f} баллов")

## Задание 2

5 баллов
Добейтесь accuracy на валидации не менее 0.84. В этом задании делать ресайз и использовать претрейн можно.

Для того чтобы выбить скор (считается ниже) на 2.5/5 балла (то есть половину за задание) достаточно соблюдать пару простых жизненных правил:
1. Аугментация (без нее сложно очень будет)
2. Оптимайзеры можно (и нужно) использовать друг с другом. Однако когда что-то проверяете, то не меняйте несколько параметров сразу - собьете логику экспериментов
3. Не используйте полносвязные модели или самые первые сверточные, используйте более современные архитектуры (что на лекциях встречались или можете пойти дальше).
4. Попробуйте сначала посмотреть качество исходной модели без дообучения, сохраните как baseline. Отсюда поймете какие слои нужно дообучать.
5. Посмотреть все ноутбуки прошедших семинаров и слепить из них что-то общее. Семинарских тетрадок хватит сверх

### Датасет

Кажется, что нужное качество можно достичь без аугментаций, поэтому будем просто применять предобученную модель ко всем картинкам до обучения с сохранением результата

In [ ]:
class FeaturesDataset(torchdata.StackDataset):
    def __init__(
            self,
            extractor_name: str,
            dataset: torchdata.Dataset,
            batch_size: int = 256,
            extractor_device: torch.device = device
        ):
        self.name = 'Features for {}'.format(dataset.name if hasattr(dataset, 'name') else 'undefined')

        # If it is already a dataset of features, return
        if isinstance(dataset, FeaturesDataset):
            return super().__init__(dataset)

        # https://github.com/pytorch/vision/issues/7744
        def get_state_dict(self, *args, **kwargs):
            kwargs.pop("check_hash")
            return torch.hub.load_state_dict_from_url(self.url, *args, **kwargs)
        torchvision.models._api.WeightsEnum.get_state_dict = get_state_dict

        # Load a pretrained model
        self.weights = torchvision.models.get_model_weights(extractor_name).DEFAULT
        self.extractor = torchvision.models.get_model(extractor_name, weights = self.weights)
        self.transform = self.weights.transforms()
        self.extractor_device = extractor_device

        # Remove last layer
        if hasattr(self.extractor, 'fc'): self.extractor.fc = torch.nn.Identity() # ResNet
        elif hasattr(self.extractor, 'classifier'): self.extractor.classifier = torch.nn.Identity() # EfficientNet
        
        save_transform = dataset.transform if hasattr(dataset, 'transform') else None
        dataset.transform = None
        assert isinstance(dataset[0][0], PIL.Image.Image) # Without transforms it should return raw images
        dataset.transform = self.transform # Use transforms for pretrained model

        targets = [ ]
        features = [ ]
        self.extractor.to(self.extractor_device).eval() # Enter evaluation mode
        loader = torchdata.DataLoader(dataset, batch_size = batch_size) # Create a dataloader
        for images_batch, targets_batch in tqdm.tqdm(loader, desc = self.name):
            with torch.no_grad():
                # Calculate features
                features_batch = self.extractor(images_batch.to(self.extractor_device))
                features.append(features_batch.detach().cpu().flatten(start_dim = 1))
                targets.append(targets_batch)

        # Free up the GPU
        self.extractor = self.extractor.to('cpu')
        if self.extractor_device.type == 'cuda': torch.cuda.empty_cache()
        elif self.extractor_device.type == 'mps': torch.mps.empty_cache()

        dataset.transform = save_transform # Restore transforms of the base dataset
        super().__init__(torch.cat(features), torch.cat(targets)) # Initialize StackDataset

In [ ]:
# Make datasets without transforms to calculate image features
train_set = MysteriousDataset(train = True, preload = True, precalculate_transform = True, transform = None)
test_set = MysteriousDataset(train = False, preload = True, precalculate_transform = True, transform = None)

### Попробуем различные модели

In [ ]:
models = [
    ("alexnet", 1024),
    ("convnext_base", 128),
    ("densenet201", 256),
    ("efficientnet_b4", 64),
    ("efficientnet_v2_s", 128),
    ("googlenet", 512),
    ("inception_v3", 256),
    ("mnasnet1_3", 512),
    ("mobilenet_v2", 256),
    ("mobilenet_v3_large", 512),
    ("regnet_x_8gf", 128),
    ("resnet152", 256),
    ("resnext101_64x4d", 128),
    ("shufflenet_v2_x2_0", 512),
    ("vgg19_bn", 64),
    ("wide_resnet101_2", 128)
]
for index, (extractor_name, batch_size) in enumerate(models):
    print("{} ({}/{}):".format(extractor_name.upper(), index + 1, len(models)))

    fix_random()
    train_features = FeaturesDataset(extractor_name, train_set, batch_size = batch_size)
    test_features = FeaturesDataset(extractor_name, test_set, batch_size = batch_size)
    model = torch.nn.Sequential(torch.nn.LazyLinear(1024), torch.nn.GELU(), torch.nn.Linear(1024, 200))
    Classifier(extractor_name, model, learning_rate = 3e-5).fit(train_features, test_features, silent = True)

Видно, что среди рассмотренных моделей resnext101_64x4d показывает наилучшее качество и достигает требуемых 84%. Попробуем немного улучшить параметры модели, чтобы она достигала требуемого качества более уверенно

### Эксперименты

#### Датасеты

In [ ]:
fix_random()
train_features = FeaturesDataset("resnext101_64x4d", train_set)
test_features = FeaturesDataset("resnext101_64x4d", test_set)

#### Количество слоев

In [ ]:
fix_random()
model = torch.nn.Sequential(torch.nn.LazyLinear(1024), torch.nn.GELU(), torch.nn.Linear(1024, 200))
Classifier('1024 + 200', model, learning_rate = 3e-5).fit(train_features, test_features, 100).summary()

In [ ]:
fix_random()
model = torch.nn.LazyLinear(200)
Classifier('200', model, learning_rate = 3e-5).fit(train_features, test_features, 100).summary()

In [ ]:
fix_random()
model = torch.nn.Sequential(torch.nn.LazyLinear(1024), torch.nn.GELU(), torch.nn.Linear(1024, 512), torch.nn.GELU(), torch.nn.Linear(512, 200))
Classifier('1024 + 512 + 200', model, learning_rate = 3e-5).fit(train_features, test_features, 100).summary()

Наибольшего качества достигает модель с двумя полносвязными слоями с 1024 и 200 нейронами

#### Нормализация

In [ ]:
fix_random()
model = torch.nn.Sequential(torch.nn.LazyLinear(1024), torch.nn.BatchNorm1d(1024), torch.nn.GELU(), torch.nn.Linear(1024, 200))
Classifier('1024 + BatchNorm + 200', model, learning_rate = 3e-5).fit(train_features, test_features, 100).summary()

Добавление нормализации ускоряет обучение (модель быстрее доходит до качества, близкого к максимальному), но повышения качества не наблюдается. Тем не менее явно наблюдается переобучение

#### Dropout

Попробуем бороться с переобучением

In [ ]:
fix_random()
model = torch.nn.Sequential(torch.nn.LazyLinear(1024), torch.nn.GELU(), torch.nn.Dropout(0.5), torch.nn.Linear(1024, 200))
Classifier('1024 + Dropout + 200', model, learning_rate = 3e-5).fit(train_features, test_features, 100).summary()

Стало немного лучше

#### ReLU

Попробуем другую функцию активации, которая используется в resnext101_64x4d

In [ ]:
fix_random()
model = torch.nn.Sequential(torch.nn.LazyLinear(1024), torch.nn.ReLU(), torch.nn.Dropout(0.5), torch.nn.Linear(1024, 200))
Classifier('1024 + Dropout + 200, ReLU', model, learning_rate = 3e-5).fit(train_features, test_features, 100).summary()

Стало немного лучше

### Итоговая модель

Лучшее качество достигается на 93 эпохе. Возьмём эту модель в качестве итоговой.

In [ ]:
fix_random()
model = torch.nn.Sequential(torch.nn.LazyLinear(1024), torch.nn.ReLU(), torch.nn.Dropout(0.5), torch.nn.Linear(1024, 200))
final_model = Classifier('Task 2 final', model, learning_rate = 3e-5).fit(train_features, test_features, 93)

In [ ]:
accuracy = final_model.calc_metrics(test_features)['Accuracy']
print(f"Accuracy: {accuracy * 100}%")
print(f"Оценка за это задание составит {numpy.clip(10 * (accuracy - 0.5) / 0.34, 0, 10):.2f} баллов")

## Отчёт об экспериментах 

### Задание 1
В ходе экспериментов было выявлено, что наибольшего качества достигает модель, поле восприятия которой равно размеру картинки: в этом случае модель учитывает всю предоставленную информацию ("видит" всю картинку, а не ее части), но и не уменьшает картинку слишком сильно (что происходит при слишком большом поле восприятия). С учетом этого была построена достаточно простая модель, состоящая из 6 сверточных слоев с размерами ядер 3x3, между некоторыми из которых происходит применение макс-пулинга размера 2x2. Такая модель верно предсказывает класс для почти 47% объектов (что уверенно превышает требуемое значение 44%).

### Задание 2